In [4]:
# !pip install moviepy
# !pip install google-generativeai
# !pip install elevenlabs
# !pip install python-dotenv
# !pip install requests
# !pip install mutagen
# !pip install pygame

In [5]:
import os
import time
import requests
import uuid
from requests.auth import HTTPBasicAuth
from elevenlabs.client import ElevenLabs
from dotenv import load_dotenv
from IPython.display import Audio, display
import pygame
from mutagen.mp3 import MP3
import google.generativeai as genai

import warnings
warnings.filterwarnings("ignore")

In [6]:
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")
VOICE_ID = os.getenv("VOICE_ID")
GEMINI_LLM_MODEL = os.getenv("GEMINI_LLM_MODEL")
LOL_LOCKFILE_PATH = os.getenv("LOL_LOCKFILE_PATH")
POLL_INTERVAL = os.getenv("POLL_INTERVAL")

gemini_api_url = f"https://generativelanguage.googleapis.com/v1beta/models/{GEMINI_LLM_MODEL}:generateContent?key={GEMINI_API_KEY}"

In [7]:
print(ELEVENLABS_API_KEY)

sk_3d93f620a58fd5526edfc7fb2157e046d8c933ae9c1fc560


In [8]:
def read_lockfile(path=LOL_LOCKFILE_PATH):
    with open(path, "r") as f:
        name, pid, port, password, protocol = f.read().split(":")
    return port, password

port, password = read_lockfile()
lcu_auth = HTTPBasicAuth("riot", password)

In [9]:
def lcu_request(endpoint):
    """Generic GET request to LCU"""
    lcu_url = f"https://127.0.0.1:{port}{endpoint}"
    resp = requests.get(lcu_url, auth=lcu_auth, verify=False)
    return resp.json()

def live_request(endpoint):
    """Generic GET request to Live Client API"""
    url = f"https://127.0.0.1:2999{endpoint}"
    resp = requests.get(url, verify=False)
    return resp.json()

In [10]:
def get_current_summoner():
    return lcu_request("/lol-summoner/v1/current-summoner")

def get_gameflow_phase():
    return lcu_request("/lol-gameflow/v1/gameflow-phase")

def get_champselect_session():
    return lcu_request("/lol-champ-select/v1/session")

def get_active_player():
    return live_request("/liveclientdata/activeplayer")

def get_player_list():
    return live_request("/liveclientdata/playerlist")

def get_game_stats():
    return live_request("/liveclientdata/gamestats")

def get_event_data():
    return live_request("/liveclientdata/eventdata")

ALL_ENDPOINTS = {
    "LCU": {
        "Current Summoner": get_current_summoner,
        "Gameflow Phase": get_gameflow_phase,
        "Champ Select Session": get_champselect_session,
    },
    "Live Client": {
        "Active Player": get_active_player,
        "Player List": get_player_list,
        "Game Stats": get_game_stats,
        "Event Data": get_event_data
    },
}

for category, endpoints in ALL_ENDPOINTS.items():
    print(f"\n==== {category} Endpoints ====")
    for name, func in endpoints.items():
        try:
            print(f"\n--- {name} ---")
            data = func()
            print(data)
        except Exception as e:
            print(f"Failed to fetch {name}: {e}")


==== LCU Endpoints ====

--- Current Summoner ---
{'accountId': 238448945, 'displayName': '', 'gameName': 'CritHappens', 'internalName': '', 'nameChangeFlag': False, 'percentCompleteForNextLevel': 74, 'privacy': 'PUBLIC', 'profileIconId': 6623, 'puuid': '24fd96a4-0a74-5c94-8438-f56d5bfa9ab6', 'rerollPoints': {'currentPoints': 500, 'maxRolls': 2, 'numberOfRolls': 2, 'pointsCostToRoll': 250, 'pointsToReroll': 0}, 'summonerId': 105732387, 'summonerLevel': 138, 'tagLine': '6969', 'unnamed': False, 'xpSinceLastLevel': 2447, 'xpUntilNextLevel': 3264}

--- Gameflow Phase ---
InProgress

--- Champ Select Session ---
{'errorCode': 'RPC_ERROR', 'httpStatus': 404, 'implementationDetails': {}, 'message': 'No active delegate'}

==== Live Client Endpoints ====

--- Active Player ---
{'abilities': {'E': {'abilityLevel': 0, 'displayName': 'Primal Howl', 'id': 'WarwickE', 'rawDescription': 'GeneratedTip_Spell_WarwickE_Description', 'rawDisplayName': 'GeneratedTip_Spell_WarwickE_DisplayName'}, 'Passive

In [11]:
class LeagueCommentator:
    def __init__(self):
        genai.configure(api_key=GEMINI_API_KEY)
        self.system_prompt = """
            You are a professional League of Legends esports commentator. 
            Your job is to provide an exciting and engaging play-by-play commentary.
            Use a vibrant and energetic tone. Focus on the most important events like kills, objectives taken (Dragons, Barons, Towers), and teamfights.
            Keep your commentary concise and impactful. Do not state that you are an AI model.
            Don't use any special caracters like *
        """
        self.model = genai.GenerativeModel(
            model_name=GEMINI_LLM_MODEL,
            system_instruction=self.system_prompt
        )
        self.chat = self.model.start_chat()


    def get_caption_from_gemini(self, event_or_context_text):
        user_prompt = f"The following events just happened in the game:\n{event_or_context_text}\n\nProvide commentary based only on the major events, make it brief."
        try:
            # Send the user message to the chat session
            response = self.chat.send_message(user_prompt)
            return response.text
        except Exception as e:
            print(f"API call failed: {e}")
            return "Our commentator seems to be having a technical issue. Please stand by."

In [12]:
if ELEVENLABS_API_KEY and VOICE_ID:
    elevenlabs_client = ElevenLabs(api_key=ELEVENLABS_API_KEY)
else:
    print("Warning: ElevenLabs API key or voice ID not found. Audio generation will be skipped.")

def get_audio_from_elevenlabs(text_to_speak):
    """
    Generates audio, saves it, plays it, and sleeps for its duration.
    """
    audio = elevenlabs_client.text_to_speech.convert(
        text=text_to_speak,
        voice_id=VOICE_ID,
        model_id="eleven_flash_v2",
        output_format="mp3_44100_128",
    )
    
    filename = str(uuid.uuid4())
    save_file_path = f"{filename}.mp3"

    with open(save_file_path, "wb") as f:
        for chunk in audio:
            if chunk:
                f.write(chunk)
    
    # Get the audio duration using mutagen
    try:
        audio_file = MP3(save_file_path)
        audio_duration = audio_file.info.length
    except Exception as e:
        print(f"Error getting audio duration: {e}")
        audio_duration = 0 

    # Play the audio
    #display(Audio(filename=save_file_path, autoplay=True))

    # Initialize the pygame mixer
    pygame.mixer.init()
    
    # Load and play the audio file
    pygame.mixer.music.load(save_file_path)
    pygame.mixer.music.play()

    # Sleep for the duration of the audio
    if audio_duration > 0:
        time.sleep(audio_duration)

    # Optional: Clean up and quit the mixer
    pygame.mixer.music.stop()
    pygame.mixer.quit()
    os.remove(save_file_path)

In [13]:
class LoLContext:
    """Stores game context for narration"""
    def __init__(self):
        self.seen_event_ids = set()
        self.champ_select_done = False
        self.players_info = []
        self.teams_info = {}
    
    def update_champ_select(self):
        try:
            session = get_champselect_session()
            self.players_info = session.get("myTeam", [])
            self.teams_info = {
                "myTeam": session.get("myTeam", []),
                "theirTeam": session.get("theirTeam", [])
            }
            if self.players_info:
                self.champ_select_done = True
        except:
            pass

    def get_new_events(self):
        events = []
        try:
            all_events = get_event_data().get("Events", [])
            for e in all_events:
                if e["EventID"] not in self.seen_event_ids:
                    self.seen_event_ids.add(e["EventID"])
                    events.append(e)
        except:
            pass
        return events

In [14]:
def event_to_text(event):
    etype = event["EventName"]
    t = int(event.get("EventTime",0))
    mm, ss = divmod(t, 60)

    if etype == "GameStart":
        desc = "The game has started!"
    elif etype == "MinionsSpawning":
        desc = "Minions have spawned."
    elif etype == "ChampionKill":
        desc = f'{event.get("KillerName","?")} killed {event.get("VictimName","?")}'
    elif etype == "TurretKilled":
        desc = f'{event.get("KillerName","?")} destroyed {event.get("TurretKilled","a turret")}'
    elif etype == "DragonKill":
        desc = f'{event.get("KillerName","?")} killed a {event.get("DragonType","dragon")} dragon'
    elif etype == "BaronKill":
        desc = f'{event.get("KillerName","?")} killed Baron Nashor'
    else:
        desc = etype

    return f"[{mm:02d}:{ss:02d}] {desc}"

In [15]:
def process_player_data(player_data):
    """
    Extracts and formats relevant player data for LLM commentary.

    This function takes a list of player JSON objects from the LCU API and
    processes it into a concise, human-readable string. This string can then
    be used to provide context to a language model for generating dynamic commentary.

    Args:
        player_data (list): A list of dictionaries, where each dictionary
                            contains data for a single player.

    Returns:
        str: A formatted string summarizing the key details for each player,
             or an empty string if the data is invalid.
    """
    if not isinstance(player_data, list) or not player_data:
        return ""

    commentary_string = ""
    for player in player_data:
        # Extract core information, using default values to avoid errors
        champion_name = player.get('championName', 'Unknown Champion')
        summoner_name = player.get('summonerName', 'Unknown Summoner')
        team = player.get('team', 'Unknown Team')
        position = player.get('position', 'NONE')
        level = player.get('level', 0)
        
        # Extract scores, handling potential missing keys
        scores = player.get('scores', {})
        kills = scores.get('kills', 0)
        deaths = scores.get('deaths', 0)
        assists = scores.get('assists', 0)

        # Extract rune and summoner spell details
        runes = player.get('runes', {})
        keystone_rune = runes.get('keystone', {}).get('displayName', 'Unknown Keystone')
        primary_tree = runes.get('primaryRuneTree', {}).get('displayName', 'Unknown Rune Tree')
        
        spells = player.get('summonerSpells', {})
        spell_one = spells.get('summonerSpellOne', {}).get('displayName', 'Unknown Spell')
        spell_two = spells.get('summonerSpellTwo', {}).get('displayName', 'Unknown Spell')

        # Create a formatted string for a single player
        player_summary = (
            f"Player: {summoner_name} ({champion_name}) on team {team}.\n"
            f"Role: {position}.\n"
            f"Scores: {kills}/{deaths}/{assists}, Level: {level}.\n"
            f"Keystone Rune: {keystone_rune} ({primary_tree} tree).\n"
            f"Summoner Spells: {spell_one} and {spell_two}.\n\n"
        )
        
        commentary_string += player_summary

    return commentary_string

def process_active_player_data(active_player_data):
    """
    Extracts and formats relevant active player data for LLM commentary.

    This function takes a single active player JSON object from the LCU API and
    processes it into a detailed, human-readable string. This string provides
    in-depth context about the player's current stats, abilities, and build
    which is invaluable for generating rich commentary.

    Args:
        active_player_data (dict): A dictionary containing data for the active player.

    Returns:
        str: A formatted string summarizing the key details for the active player,
             or an empty string if the data is invalid.
    """
    if not isinstance(active_player_data, dict) or not active_player_data:
        return ""

    # Extract core information
    summoner_name = active_player_data.get('summonerName', 'Unknown Summoner')
    champion_stats = active_player_data.get('championStats', {})
    level = active_player_data.get('level', 0)
    current_gold = active_player_data.get('currentGold', 0)

    # Extract champion stats with default values
    current_health = champion_stats.get('currentHealth', 0)
    max_health = champion_stats.get('maxHealth', 0)
    attack_damage = champion_stats.get('attackDamage', 0)
    ability_power = champion_stats.get('abilityPower', 0)
    armor = champion_stats.get('armor', 0)
    magic_resist = champion_stats.get('magicResist', 0)
    move_speed = champion_stats.get('moveSpeed', 0)
    
    # Extract rune information
    full_runes = active_player_data.get('fullRunes', {})
    keystone_rune = full_runes.get('keystone', {}).get('displayName', 'Unknown Keystone')
    primary_tree = full_runes.get('primaryRuneTree', {}).get('displayName', 'Unknown Rune Tree')
    
    # Extract abilities
    abilities = active_player_data.get('abilities', {})
    q_ability = abilities.get('Q', {}).get('displayName', 'Q Ability')
    w_ability = abilities.get('W', {}).get('displayName', 'W Ability')
    e_ability = abilities.get('E', {}).get('displayName', 'E Ability')
    r_ability = abilities.get('R', {}).get('displayName', 'R Ability')

    # Create a formatted string for the active player
    player_summary = (
        f"Active Player: {summoner_name} (Level {level})\n"
        f"Health: {current_health:.1f}/{max_health:.1f}\n"
        f"Gold: {current_gold:.1f}\n"
        f"Stats:\n"
        f" - AD: {attack_damage:.1f}, AP: {ability_power:.1f}\n"
        f" - Armor: {armor:.1f}, Magic Resist: {magic_resist:.1f}\n"
        f" - Move Speed: {move_speed:.1f}\n"
        f"Runes: {keystone_rune} ({primary_tree} tree)\n"
        f"Abilities:\n"
        f" - Q: {q_ability}\n"
        f" - W: {w_ability}\n"
        f" - E: {e_ability}\n"
        f" - R: {r_ability}\n"
    )

    return player_summary

In [16]:
def main_loop():
    is_first_run = True
    ctx = LoLContext()
    lolCommentator = LeagueCommentator()
    while True:
        # Handle the initial welcome message on the first run
        if is_first_run:
            is_first_run = False
            intro = "Welcome, everyone, to the ultimate battleground where legends are made! I'm your host, bringing you the fastest plays and sharpest calls from today's high-stakes tournament. Get ready for insane strategies and jaw-dropping action as our top contenders prove they're the best in the game."
            print(intro)
            get_audio_from_elevenlabs(intro)
        phase = get_gameflow_phase()
        # 1. Pregame: champ select
        if phase in ["Lobby", "Matchmaking", "ChampSelect"] and not ctx.champ_select_done:
            ctx.update_champ_select()
            if ctx.champ_select_done:
                text = "Champ select is done. Teams and bans are set."
                caption = lolCommentator.get_caption_from_gemini(text)
                print(caption)
                get_audio_from_elevenlabs(caption)
        
        # 2. In-game: fetch events
        if phase == "InProgress":
            context = ""
            new_events = ctx.get_new_events()
            for e in new_events:
                text = event_to_text(e)
                context = context + text + "\n"
            if context:  # Only generate a caption if there are new events
                caption = lolCommentator.get_caption_from_gemini(text)
                print(caption)
                get_audio_from_elevenlabs(caption)
            else: 
                commentary_string_from_player_list = process_player_data(get_player_list())
                commentary_string_from_active_data = process_active_player_data(get_active_player())
                caption = lolCommentator.get_caption_from_gemini(commentary_string_from_player_list + "\n" + commentary_string_from_active_data)
                print(caption)
                get_audio_from_elevenlabs(caption)
                
        
        # time.sleep(int(POLL_INTERVAL))

In [17]:
# ======== RUN ========
print("🚀 Starting AI commentator...")
main_loop()

🚀 Starting AI commentator...
Welcome, everyone, to the ultimate battleground where legends are made! I'm your host, bringing you the fastest plays and sharpest calls from today's high-stakes tournament. Get ready for insane strategies and jaw-dropping action as our top contenders prove they're the best in the game.
AND FIRST BLOOD GOES TO TEAM ALPHA! What a chaotic dive in the bot lane, Team Alpha comes out on top! They've drawn first blood and are looking to snowball this early advantage!


KeyboardInterrupt: 